In [1]:
from selenium import webdriver as webDriver
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import time
import re
import csv
import datetime

def catch(url):
    try_flag = True
    while(try_flag):
        try:
            html = urlopen(url)
            soup = BeautifulSoup(html, 'html.parser')
            #print(soup.prettify())
            try_flag = False
            return soup
        except Exception as e:
            print(e)
            
def catch_sel(url):
    try_flag = True
    while(try_flag):
        try:
            driver.get(url)
            try_flag = False
        except Exception as e:
            print(e)
            driver.refresh()

flag = True
page = START = 1 ### 시작 페이지
END = 2 ### 끝 페이지 - 1

TITLE, GENRE, DEV, PUB, RED, USER_SCORE, SCORE, TAG, URL, FR = [], [], [], [], [], [], [], [], [], []

firefox_dir = r'C:\Users\ghc11\Desktop\geckodriver-v0.28.0-win64\geckodriver.exe'
driver = webDriver.Firefox(executable_path=firefox_dir)
#driver.get("https://store.steampowered.com/search/?sort_by=Relevance_ASC&page=1")

while(flag):
    print("now >>>>> (" + str(page) + ") in: " + str(datetime.datetime.now()))
    
    soup = catch("https://store.steampowered.com/search/?sort_by=_ASC&page=" + str(page))
            
    #driver.get("https://store.steampowered.com/search/?term=ea+play&page=1")
    
    #url = 'https://store.steampowered.com/search/?sort_by=_ASC&page=' + str(page)
    #headers = {'User-Agent': 'Mozilla/5.0'}
    #request = Request(url, headers = headers)
    
    #html = urlopen(request)
    #soup = BeautifulSoup(html, 'html.parser')
    #soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    while(1):
        href = soup.find('div', {'id':"search_resultsRows"})
        if href != None:
            print("href Pass!")
            break;
        else:
            print("href Error!")
            #url = 'https://store.steampowered.com/search/?sort_by=_ASC&page=' + str(page)
            #headers = {'User-Agent': 'Mozilla/5.0'}
            #request = Request(url, headers = headers)

            #html = urlopen(request)
            
            soup = catch("https://store.steampowered.com/search/?sort_by=Relevance_ASC&page=" + str(page))
        
    href2 = href.find_all('span', 'title')

    for i in href2:
        TITLE.append(i.text)
    
    href3 = href.find_all('a', href=True)

    for i in href3:
        temp_flag = False
        
        URL.append(i['href'])
        
        url = i['href']
        #headers = {'User-Agent': 'Mozilla/5.0'}
        #request = Request(url, headers = headers)
        
        soup = catch(url)
        print(url)
        
        #html = urlopen(request)
        #soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        count = 0
        
        while(1):
            detail = soup.find_all('div', 'block_content_inner')
            checker = True
            if len(detail) != 0:
                #print("detail Pass!")
                for i in detail:
                    if i.find('div', 'details_block') != None:
                        detail = i.find('div', 'details_block')
                        checker = False
                        break;
                if not checker:
                    break;
                    
            else:
                #print(" href Error!")
                #url = i['href']
                #headers = {'User-Agent': 'Mozilla/5.0'}
                #request = Request(url, headers = headers)

                #html = urlopen(request)
                #soup = BeautifulSoup(html, 'html.parser')
                
                catch_sel(url)
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                
                if soup.find('div', 'SaleSectionCtn') != None:
                    GENRE.append(" ")
                    DEV.append(" ")
                    PUB.append(" ")
                    RED.append(" ")
                    USER_SCORE.append(" ")
                    SCORE.append(" ")
                    TAG.append(" ")
                    FR.append(" ")
                    temp_flag = True
                    break;

                if soup.find('div', 'agegate_birthday_selector') != None:
                    #print("19+ Detected")
                    driver.find_element_by_xpath('''//*[@id="ageDay"]''').send_keys('9')
                    driver.find_element_by_xpath('''//*[@id="ageMonth"]''').send_keys('April')
                    driver.find_element_by_xpath('''//*[@id="ageYear"]''').send_keys('1997')
                    driver.find_element_by_class_name('''btnv6_blue_hoverfade''').click()
                    time.sleep(5)
                    soup = BeautifulSoup(driver.page_source, 'html.parser')
                
                else:
                    count += 1
                    driver.refresh()
                    soup = BeautifulSoup(driver.page_source, 'html.parser')
                    
                    if count > 20:
                        GENRE.append(" ")
                        DEV.append(" ")
                        PUB.append(" ")
                        RED.append(" ")
                        USER_SCORE.append(" ")
                        SCORE.append(" ")
                        TAG.append(" ")
                        FR.append(" ")
                        temp_flag = True
                        break;
                    #soup = catch(url)
        
        
        if temp_flag:
            continue;
        #detail = soup.find('div', 'block_content_inner').find('div', 'details_block')
        temp = re.split(r'[\n:]', (detail.text.strip()))

        temp = [i for i in temp if i]
        temp2 = []

        for i in temp:
            temp2.append(i.strip())

        if 'Franchise' in temp2:
            FR.append(temp2[temp2.index('Franchise') + 1])
        else:
            FR.append(" ")
        
        if 'Genre' in temp2:
            GENRE.append(temp2[temp2.index('Genre') + 1])
        else:
            GENRE.append(" ")
            
        if 'Developer' in temp2:
            DEV.append(temp2[temp2.index('Developer') + 1])
        else:
            DEV.append(" ")
        
        if 'Publisher' in temp2:
            PUB.append(temp2[temp2.index('Publisher') + 1])
        else:
            PUB.append(" ")
        
        if 'Release Date' in temp2:
            RED.append(temp2[temp2.index('Release Date') + 1])
        else:
            RED.append(" ")

        #print(": ".join(temp2))
        
        if soup.find('span', 'game_review_summary') != None:
            USER_SCORE.append(soup.find('span', 'game_review_summary').text)
        else:
            USER_SCORE.append(" ")

        raw_TAG = []
        
        if soup.find('div', 'glance_tags') != None:
            user_tag = soup.find('div', 'glance_tags').find_all('a', href=True)
            for i in user_tag:
                raw_TAG.append(i.text.strip())

            TAG.append(", ".join(raw_TAG))
        else:
            TAG.append(" ")

        if soup.find('div', {'id':'game_area_metascore'}) != None and soup.find('div', {'id':'game_area_metascore'}).find('div', 'score') != None:
            #print("test")
            SCORE.append(soup.find('div', {'id':'game_area_metascore'}).find('div', 'score').text.strip())
        else:
            SCORE.append(" ")
    
    page += 1
    
    if(page == END):
        flag = False
        
print("end in: " + str(datetime.datetime.now()))

csvFile = open('test_' + str(START) + "_" + str(END) + '.csv', 'w+', newline='', encoding="utf-8-sig")

try:
    writer = csv.writer(csvFile)
    writer.writerow(('Name', 'Genre', 'Developer', 'Publisher', 'Franchise', 'Release Date', 'User score', 'Meta score', 'User tag', 'Url'))
    for Title, Genre, Dev, Pub, Fr, Red, User_score, Score, Tag, Url in zip(TITLE, GENRE, DEV, PUB, FR, RED, USER_SCORE, SCORE, TAG, URL):
        writer.writerow((Title, Genre, Dev, Pub, Fr, Red, User_score, Score, Tag, Url))
finally:
    csvFile.close()

now >>>>> (1) in: 2020-12-04 18:51:05.105986
href Pass!
https://store.steampowered.com/app/730/CounterStrike_Global_Offensive/?snr=1_7_7_230_150_1
https://store.steampowered.com/app/1085660/Destiny_2/?snr=1_7_7_230_150_1
https://store.steampowered.com/app/359550/Tom_Clancys_Rainbow_Six_Siege/?snr=1_7_7_230_150_1
https://store.steampowered.com/app/1263850/Football_Manager_2021/?snr=1_7_7_230_150_1
https://store.steampowered.com/app/1172470/Apex_Legends/?snr=1_7_7_230_150_1
https://store.steampowered.com/app/578080/PLAYERUNKNOWNS_BATTLEGROUNDS/?snr=1_7_7_230_150_1
https://store.steampowered.com/app/381210/Dead_by_Daylight/?snr=1_7_7_230_150_1
https://store.steampowered.com/app/271590/Grand_Theft_Auto_V/?snr=1_7_7_230_150_1
https://store.steampowered.com/app/739630/Phasmophobia/?snr=1_7_7_230_150_1
https://store.steampowered.com/app/594570/Total_War_WARHAMMER_II/?snr=1_7_7_230_150_1
https://store.steampowered.com/app/1174180/Red_Dead_Redemption_2/?snr=1_7_7_230_150_1
https://store.steampo